## TCC CBB

Desenvolvido por Ricardo e Thyago.

#### Objetivo: 

Aplicação das técnicas de classificação SVM, MLP e Tensorflow junto ao algoritmo bio-inspirado PSO e aplicado na base de dados de doenças cardiovasculares.

Neste script é aplicado os algoritmos de classificação SVM, MLP e Tensorflow e o modelo estatístico
com seu ponderamento feito pelos algoritmos Bio-inspirados.

Fez-se a analise dos parâmetros de aplicação do XGBoost e a técnica foi reaplicada com os paramêtros atualizados.

### APLICAÇÃO COM A BASE DE DADOS CUSTUMER

In [ ]:
import random
import bigml.api
import os
import pandas as pd
from bigml.api import BigML
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection, metrics
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

### import SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
def aplicaSVM(X_train, X_test, y_train): 
    
    model_svm = SVC(probability = True, kernel = 'rbf', gamma = 'scale')
    model_svm.fit(X_train, y_train)
    predictionsSVM = model_svm.predict_proba(X_test)
    
    return predictionsSVM

### Import do KERAS

In [ ]:
#from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
#tensorflow.keras.models
tf.keras.backend.clear_session()  # For easy reset of notebook state.
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
def aplicaTensor(X_train, X_test, y_train, y_test, dim_input):
    model = Sequential()
    model.add(Dense(40, input_dim=dim_input, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=30, batch_size=100)
    
    predictionsTensor = model.predict_proba(X_test)

    return predictionsTensor

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['animation.ffmpeg_path'] = 'C:\\Users\\Thyago M\\Desktop\\ffmpeg.exe'

### import do MLP

In [ ]:
from sklearn import linear_model, model_selection, metrics
from sklearn.neural_network import MLPClassifier

In [ ]:
def aplicaMLP(X_train, X_test, y_train):
    #X_train, X_test, y_train, y_test = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=0)
   
    modelMLP = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    modelMLP.fit(X_train, y_train)
    predictionsMLP = modelMLP.predict_proba(X_test)
    
    return predictionsMLP

# BASE DE DADOS

In [ ]:
def converte_binario(palavra):
    if palavra=='Yes' or palavra==True:
        return 0
    elif palavra=='No' or palavra==False:
        return 1

In [ ]:
def descarregaBaseCardio():    
    df_cardio = pd.read_csv(os.getcwd()+'\\cardio_train.csv',sep=";")
    df_cardio_target = df_cardio['cardio']
    df_cardio.drop('cardio', axis=1, inplace=True)
    df_cardio.drop('id', axis=1, inplace=True)
    
    return df_cardio, df_cardio_target

## APLICA K-FOLD

In [ ]:
def aplicaKFold(x_pca, df_weather_target):
    divisao = 0.2
    X_train, X_test, y_train, y_test = train_test_split(x_pca, df_weather_target, test_size=divisao)
   
    return (X_train, X_test, y_train, y_test)

In [ ]:
def verifica_acertos(predicoes, y_test):
    qtd_acertos = []
    for i in range(predicoes.shape[0]):
        if predicoes[i] == y_test[i]:
            qtd_acertos.append(1)
        else:
            qtd_acertos.append(0)
    return np.asarray(qtd_acertos)

In [ ]:
def aplicaKFoldVariado(x_pca, df_cardio_target,divisao):
    X_train, X_test, y_train, y_test = train_test_split(x_pca, df_cardio_target, test_size=divisao)
    return (X_train, X_test, y_train, y_test)

# Bio Inspirados

### Classe SW

In [ ]:
class sw(object):

    def __init__(self):

        self.__Positions = []
        self.__Gbest = []

    def _set_Gbest(self, Gbest):
        self.__Gbest = Gbest

    def _points(self, agents):
        self.__Positions.append([list(i) for i in agents])

    def get_agents(self):
        """Returns a history of all agents of the algorithm (return type:
        list)"""

        return self.__Positions

    def get_Gbest(self):
        """Return the best position of algorithm (return type: list)"""

        return list(self.__Gbest)

## Classe FA

In [ ]:
from math import exp
import numpy as np

class fa(sw):
    """
    Firefly Algorithm
    """
    def __init__(self, n, function, lb, ub, dimension, iteration, y_test, vetor_pbest, vetor_acertos, csi=1, psi=1,
                 alpha0=10, alpha1=0.5, norm0=0, norm1=0.1):
        """
        :param n: number of agents
        :param function: test function
        :param lb: lower limits for plot axes
        :param ub: upper limits for plot axes
        :param dimension: space dimension
        :param iteration: number of iterations
        :param csi: mutual attraction (default value is 1)
        :param psi: light absorption coefficient of the medium
        (default value is 1)
        :param alpha0: initial value of the free randomization parameter alpha
        (default value is 1)
        :param alpha1: final value of the free randomization parameter alpha
        (default value is 0.1)
        :param norm0: first parameter for a normal (Gaussian) distribution
        (default value is 0)
        :param norm1: second parameter for a normal (Gaussian) distribution
        (default value is 0.1)
        """

        super(fa, self).__init__()

        self.__agents = np.random.uniform(lb, ub, (n, dimension))
        self._points(self.__agents)
        Pbest = self.__agents[np.array([function(x, y_test)
                                        for x in self.__agents]).argmax()]
        Gbest = Pbest
        vetor_pbest.append(Pbest)
        vetor_acertos.append(function(Pbest, y_test))
        for t in range(iteration):
            alpha = alpha1 + (alpha0 - alpha1) * exp(-t)
            for i in range(n):
                fitness = [function(x, y_test) for x in self.__agents]
                for j in range(n):
                    if fitness[i] < fitness[j]:
                        self.__move(i, j, t, csi, psi, alpha, dimension,
                                    norm0, norm1)
                    else:
                        self.__agents[i] += np.random.normal(norm0, norm1,
                                                             dimension)

            self.__agents = np.clip(self.__agents, lb, ub)
            self._points(self.__agents)
            
            Pbest = self.__agents[
                np.array([function(x, y_test) for x in self.__agents]).argmax()]
            if function(Pbest, y_test) > function(Gbest, y_test):
                Gbest = Pbest
            vetor_pbest.append(Gbest)
            vetor_acertos.append(function(Gbest, y_test))
        self._set_Gbest(Gbest)

    def __move(self, i, j, t, csi, psi, alpha, dimension, norm0, norm1):
        r = np.linalg.norm(self.__agents[i] - self.__agents[j])
        beta = csi / (1 + psi * r ** 2)
        self.__agents[i] = self.__agents[j] + beta * (
            self.__agents[i] - self.__agents[j]) + alpha * exp(-t) * \
                                                   np.random.normal(norm0,
                                                                    norm1,
                                                                    dimension)

### Classe SWA

In [ ]:
class swa(object):

    def __init__(self):

        self.__Positions = []
        self.__Gbest = []

    def _set_Gbest(self, __Gbest):
        self.__Gbest = __Gbest

    def _points(self, agents):
        self.__Positions.append([list(i) for i in agents])

    def get_agents(self):
        """Returns a history of all agents of the algorithm (return type:
        list)"""

        return self.__Positions

    def get_Gbest(self):
        """Return the best position of algorithm (return type: list)"""

        return list(self.__Gbest)

### Classe PSO

In [ ]:
import numpy as np

class pso(swa):
    """
    Particle Swarm Optimization
    """

    def __init__(self, n, function, lb, ub, dimension, iteration, y_test_svm, vetor_pbest, vetor_acertos, w=0.5, c1=1,
                 c2=1):
        """
        :param n: number of agents
        :param function: test function
        :param lb: lower limits for plot axes
        :param ub: upper limits for plot azes
        :param dimension: space dimension
        :param iteration: the number of iterations
        :param w: balance between the range of research and consideration for
        suboptimal decisions found (default value is 0.5):
        w>1 the particle velocity increases, they fly apart and inspect
         the space more carefully;
        w<1 particle velocity decreases, convergence speed depends
        on parameters c1 and c2 ;
        :param c1: ratio between "cognitive" and "social" component
        (default value is 1)
        :param c2: ratio between "cognitive" and "social" component
        (default value is 1)
        """

        super(pso, self).__init__()

        self.__agents = np.random.uniform(lb, ub, (n, dimension))
        velocity = np.zeros((n, dimension))
        self._points(self.__agents)

        Pbest = self.__agents[np.array([function(x, y_test_svm)
                                        for x in self.__agents]).argmax()]
        Gbest = Pbest
        vetor_pbest.append(Pbest)
        vetor_acertos.append(function(Pbest, y_test_svm))
        
        for t in range(iteration):
            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))
            velocity = w * velocity + c1 * r1 * (
                Pbest - self.__agents) + c2 * r2 * (
                Gbest - self.__agents)
            self.__agents += velocity
            self.__agents = np.clip(self.__agents, lb, ub)
            self._points(self.__agents)
         
            Pbest = self.__agents[
                np.array([function(x, y_test_svm) for x in self.__agents]).argmax()]
            if function(Pbest, y_test_svm) > function(Gbest, y_test_svm):
                Gbest = Pbest
            vetor_pbest.append(Pbest)
            vetor_acertos.append(function(Pbest, y_test_svm))
        
        self._set_Gbest(Gbest)
        print(Gbest)

## Aplicações FA

In [ ]:
#import SwarmPackagePy
import matplotlib.pyplot as plt
#from SwarmPackagePy import testFunctions as tf

df_cardio, df_cardio_target = descarregaBaseCardio()
scaler = StandardScaler()

random = np.random.randint(1,1000)
scaler.fit(df_cardio)

scaled_data = scaler.transform(df_cardio)  

pca_svm = PCA(n_components = 11)
pca_svm.fit(scaled_data)
x_pca_svm = pca_svm.transform(scaled_data)

pca_tensorflow = PCA(n_components=9)
pca_tensorflow.fit(scaled_data)
x_pca_tensorflow = pca_tensorflow.transform(scaled_data)

pca_MLP = PCA(n_components = 6)
pca_MLP.fit(scaled_data)
x_pca_mlp = pca_MLP.transform(scaled_data)

X_train_svm, X_test_svm, y_train_svm, y_test_svm = aplicaKFold2(x_pca_svm, df_cardio_target, 0.1, random)
X_train_tensorflow, X_test_tensorflow, y_train_tensorflow, y_test_tensorflow = aplicaKFold2(x_pca_tensorflow, df_cardio_target, 0.1, random)
X_train_mlp, X_test_mlp, y_train_mlp, y_test_mlp = aplicaKFold2(x_pca_mlp, df_cardio_target, 0.1, random)

predictionsSVM = aplicaSVM(X_train_svm, X_test_svm, y_train_svm)
predictionsTensorflow = aplicaTensor(X_train_tensorflow, X_test_tensorflow, y_train_tensorflow, y_test_tensorflow, 9)
predictionsMLP = aplicaMLP(X_train_mlp, X_test_mlp, y_train_mlp)

predictionsTensorflowAjustado = []

for row in predictionsTensorflow:
    predictionsTensorflowAjustado.append(row[0])
predictionsTensorflowAjustado = np.asarray(predictionsTensorflowAjustado)

vetor_pbest = []
vetor_acertos = []

alh = fa(40, retorna_acertos, 0, 1, 2, 20, y_test_svm)
#alh1 = pso(40, retorna_acertos, 0, 1, 3, 20,  y_test_tensorflow, vetor_pbest,vetor_acertos )
arr1 = np.array(alh.get_agents())
animationFA(alh.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)
animation3D(alh.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)
print(alh.get_Gbest())

## APlicação do Modelo estatístico para o FA

In [ ]:
svm_alpha = ( ( 1 - predictionsSVM[:,1]) ** alh.get_Gbest()[0] )
tensorflow_beta =  ( ( 1 - predictionsTensorflowAjustado) ** alh.get_Gbest()[1])
mlp_gamma = ( ( 1 - predictionsMLP[:,1] ) ** alh.get_Gbest()[2])
predicoes = 1 - (svm_alpha*tensorflow_beta*mlp_gamma)
qtd_acertos = np.subtract(y_test_svm,np.round(predicoes))
np.count_nonzero(qtd_acertos == 0)/len(y_test_svm) * 100

## APlicação PSO

In [ ]:
#import SwarmPackagePy
import matplotlib.pyplot as plt
#from SwarmPackagePy import testFunctions as tf

df_bigml, df_bigml_target = descarregaBaseCardio()
scaler = StandardScaler()

random = np.random.randint(1,1000)
scaler.fit(df_cardio)

scaled_data = scaler.transform(df_cardio)  

pca_svm = PCA(n_components = 11)
pca_svm.fit(scaled_data)
x_pca_svm = pca_svm.transform(scaled_data)

pca_tensorflow = PCA(n_components=9)
pca_tensorflow.fit(scaled_data)
x_pca_tensorflow = pca_tensorflow.transform(scaled_data)
#predictionsTensorflow = aplicaTensor(X_train_tensorflow, X_test_tensorflow, y_train_tensorflow, 17)

pca_MLP = PCA(n_components = 11)
pca_MLP.fit(scaled_data)
x_pca_mlp = pca_MLP.transform(scaled_data)

X_train_svm, X_test_svm, y_train_svm, y_test_svm = aplicaKFold2(x_pca_svm, df_cardio_target, 0.1, random)
X_train_tensorflow, X_test_tensorflow, y_train_tensorflow, y_test_tensorflow = aplicaKFold2(x_pca_tensorflow, df_cardio_target, 0.1, random)
X_train_mlp, X_test_mlp, y_train_mlp, y_test_mlp = aplicaKFold2(x_pca_mlp, df_cardio_target, 0.1, random)

predictionsSVM = aplicaSVM(X_train_svm, X_test_svm, y_train_svm)
predictionsTensorflow = aplicaTensor(X_train_tensorflow, X_test_tensorflow, y_train_tensorflow, y_test_tensorflow, 9)
predictionsMLP = aplicaMLP(X_train_mlp, X_test_mlp, y_train_mlp)

predictionsTensorflowAjustado = []

for row in predictionsTensorflow:
    predictionsTensorflowAjustado.append(row[0])
predictionsTensorflowAjustado = np.asarray(predictionsTensorflowAjustado)

vetor_pbest = []
vetor_acertos = []

#alh = fa(40, retorna_acertos, 0, 1, 2, 20, y_test_svm)
alh1 = pso(40, retorna_acertos, 0, 1, 3, 20,  y_test_tensorflow, vetor_pbest,vetor_acertos )
arr1 = np.array(alh1.get_agents())
animationFA(alh1.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)
animation3D(alh1.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)
print(alh1.get_Gbest())

### Aplicação do Modelo estatístico para o PSO

In [ ]:
svm_alpha = ( ( 1 - predictionsSVM[:,1]) ** alh1.get_Gbest()[0] )
tensorflow_beta =  ( ( 1 - predictionsTensorflowAjustado) ** alh1.get_Gbest()[1])
mlp_gamma = ( ( 1 - predictionsMLP[:,1] ) ** alh1.get_Gbest()[2])
predicoes = 1 - (svm_alpha*tensorflow_beta*mlp_gamma)
qtd_acertos = np.subtract(y_test_svm,np.round(predicoes))
np.count_nonzero(qtd_acertos == 0)/len(y_test_svm) * 100